In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
root_dir = '/content/drive/MyDrive/Models_R50_mish_more_layers_t2'

In [3]:
import os
import multiprocessing
import numpy as np
import cv2
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms

In [4]:
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  

device = torch.device(dev)
print(device)

cuda:0


In [5]:
!nvidia-smi

Mon Apr 26 05:44:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
#https://towardsdatascience.com/implementing-the-new-state-of-the-art-mish-activation-with-2-lines-of-code-in-pytorch-e7ef438a5ee7
def mish(x): 
  return (x * torch.tanh(nn.functional.softplus(x)))

In [7]:
#https://machinelearningmastery.com/pytorch-tutorial-develop-deep-learning-models/
class Model_3d(nn.Module):
    '''
    A model to use on 3d objects
    '''
    def __init__(self, fc_nodes, no_views, no_classes):
      super(Model_3d, self).__init__()
      self.resnet = torchvision.models.resnet50(pretrained=True) #https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

      num_features = self.resnet.fc.out_features

      self.fcs = nn.ModuleList()

      for idx in range(no_views):
        self.fcs.append(nn.Linear(num_features, fc_nodes))


      self.dense_1 = nn.Linear(fc_nodes * no_views, 5120)
      self.dense_2 = nn.Linear(5120, 2560)
      self.dense_3 = nn.Linear(2560, 1280)
      self.dense_4 = nn.Linear(1280, 512)
      self.dense_5 = nn.Linear(512, no_classes)

    
    def forward(self, x):
      features = []

      for i, fc in enumerate(self.fcs):
        sample = x[:, i, ...]
        r_out = self.resnet(sample)

        fc_out = fc(r_out)
        features.append(fc_out)
      
      stack = torch.stack(features, 1)

      reshaped = stack.view([x.shape[0], -1])

      mish_out = mish(reshaped)

      x = self.dense_1(mish_out)
      x = self.dense_2(mish(x))
      x = self.dense_3(mish(x))
      x = self.dense_4(mish(x))
      x = self.dense_5(mish(x))


      return torch.nn.functional.softmax(x)

In [8]:
class Date_3d_Cached(torch.utils.data.Dataset):
    def __init__(self, x, y):
      self.x = torch.from_numpy(x)
      self.y = torch.from_numpy(y)
    
    def __len__(self):
      return self.x.shape[0]

    def __getitem__(self, idx):
      return self.x[idx] /255.0, self.y[idx]

In [11]:
views = ['bottom', 'side_1', 'side_2', 'side_3', 'side_4', 'top']
x = np.load('/content/drive/MyDrive/Cache/x.npy')
y = np.load('/content/drive/MyDrive/Cache/y.npy')

training_data = Date_3d_Cached(x, y)

In [12]:
#90/10 train val split
train_len = int(len(training_data) * .9)
val_len = len(training_data) - train_len

In [13]:
train_set, val_set = torch.utils.data.random_split(training_data, [train_len, val_len],torch.Generator().manual_seed(42))

In [14]:
batch_size = 16

In [15]:
model = Model_3d(1024, len(views), 10)
model.to(device)
print(device)
training_loader = torch.utils.data.DataLoader(train_set, batch_size= batch_size, shuffle=True, num_workers=os.cpu_count())
val_loader = torch.utils.data.DataLoader(val_set, batch_size= batch_size, shuffle=True, num_workers=os.cpu_count())

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth



cuda:0


In [16]:
print(os.cpu_count())
print(train_len)
print(val_len)

2
3591
400


In [17]:
#https://medium.com/analytics-vidhya/saving-and-loading-your-model-to-resume-training-in-pytorch-cb687352fa61
def save_ckp(state, model_path):
    torch.save(state, model_path)

In [18]:
#https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

if not os.path.exists(root_dir):
    os.makedirs(root_dir)


low_val_loss = 30000000
best_acc = -1
epoch = 0

val_loss_path = None
acc_path = None
#https://stackoverflow.com/questions/8078330/csv-writing-within-loop
import csv
with open(os.path.join(root_dir, 'logs.csv'), 'w') as file_csv:
  writer=csv.writer(file_csv, delimiter=',',lineterminator='\n',)
  writer.writerow(['epoch', 'loss', 'acc', 'val_loss', 'val_acc'])


  for epoch in range(1, 301):
      row = [epoch]
      is_best = False
      running_loss = 0.0
      correct = 0
      total = 0

      model.train()

      t0 = time.time()
      for i, data in enumerate(training_loader):
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data
          inputs = inputs.to(device)
          labels = labels.to(device)

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = model(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          # print statistics
          correct += (outputs.argmax(1) == labels).float().sum() #https://stackoverflow.com/questions/51503851/calculate-the-accuracy-every-epoch-in-pytorch
          total += labels.shape[0]
          running_loss += loss.item()

      print('epoch', epoch)
      
      print('{} seconds'.format(time.time() - t0))
      print('loss', running_loss)
      row.append(running_loss)
      running_loss = 0.0

      accuracy = 100 * correct / total
      print("Accuracy = {}".format(accuracy))
      row.append(torch.IntTensor.item(accuracy))
      correct = 0
      total= 0

      running_loss = 0.0
      correct = 0
      total = 0

      optimizer.zero_grad()
      model.eval()
      t0 = time.time()
      with torch.no_grad():
          for i, data in enumerate(val_loader):
              
              # get the inputs; data is a list of [inputs, labels]
              inputs, labels = data
              inputs = inputs.to(device)
              labels = labels.to(device)            


              # forward + backward + optimize
              outputs = model(inputs)
              loss = criterion(outputs, labels)

              correct += (outputs.argmax(1) == labels).float().sum() #https://stackoverflow.com/questions/51503851/calculate-the-accuracy-every-epoch-in-pytorch
              total += labels.shape[0]
              running_loss += loss.item()

      print('{} seconds'.format(time.time() - t0))
      print('val loss', running_loss)
      row.append(running_loss)
      

      accuracy = 100 * correct / total
      print(" Val Accuracy = {}".format(accuracy))
      row.append(torch.IntTensor.item(accuracy))

      model_name = 'ep_' + str(epoch) + '_loss_' + str(running_loss) + '_acc_' + str(accuracy) + '.pt'
      full_model_path = os.path.join(root_dir, model_name)
      checkpoint = {
          'epoch': epoch + 1,
          'state_dict': model.state_dict(),
          'optimizer': optimizer.state_dict()
      }
      if running_loss < low_val_loss:
        if val_loss_path and os.path.exists(val_loss_path):
          os.remove(val_loss_path)
        val_loss_path = full_model_path.replace('.pt', 'val_loss_best.pt')
        save_ckp(model, val_loss_path)
        low_val_loss = running_loss
        print('new low loss')

      if accuracy > best_acc:
        if acc_path and os.path.exists(acc_path):
          os.remove(acc_path)
        acc_path = full_model_path.replace('.pt', 'acc_best.pt')
        save_ckp(model, acc_path)
        best_acc = accuracy
        print('new best acc')
      full_model_path = os.path.join(root_dir, 'last.pt')
      save_ckp(checkpoint, full_model_path)
          

      

      correct = 0
      total= 0
      running_loss = 0.0
      writer.writerow(row)
  print('Finished Training')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


epoch 1
126.20408868789673 seconds
loss 517.850583076477
Accuracy = 17.4881649017334
4.912407636642456 seconds
val loss 57.51591491699219
 Val Accuracy = 25.5
new low loss
new best acc
epoch 2
126.39545583724976 seconds
loss 517.5094499588013
Accuracy = 21.91590118408203
4.994104623794556 seconds
val loss 57.47051978111267
 Val Accuracy = 25.5
new low loss
epoch 3
126.36803197860718 seconds
loss 517.1237692832947
Accuracy = 21.91590118408203
5.006756544113159 seconds
val loss 57.41804313659668
 Val Accuracy = 25.5
new low loss
epoch 4
126.4284029006958 seconds
loss 516.6346328258514
Accuracy = 21.91590118408203
4.997793197631836 seconds
val loss 57.345847606658936
 Val Accuracy = 25.5
new low loss
epoch 5
126.12198829650879 seconds
loss 515.8288204669952
Accuracy = 21.91590118408203
4.985052585601807 seconds
val loss 57.19677114486694
 Val Accuracy = 25.5
new low loss
epoch 6
126.2257080078125 seconds
loss 510.62376499176025
Accuracy = 22.528545379638672
4.980290174484253 seconds
val l

In [ ]:
import pandas as pd
     
# dictionary of lists  
this_dict = {'training loss': training_loss_list, 'training acc': training_acc_list, 'val loss': val_loss_list, 'val acc':val_acc_list}  
       
df = pd.DataFrame(this_dict) 
    
# saving the dataframe 
df.to_csv(os.path.join(root_dir, 'logs.csv'))

In [ ]:
print('saved csv')